In [1]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -qU "langchain[mistralai]"

Note: you may need to restart the kernel to use updated packages.


In [4]:
import getpass
import os

if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("mistral-large-latest", model_provider="mistralai")

For the Embeddding -HuggingFace

In [2]:
pip install -qU langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")

For the Vector Databse I  Use the FAISS from META

In [2]:
pip install -qU langchain-community

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS(embedding_function=embeddings)

TypeError: FAISS.__init__() missing 3 required positional arguments: 'index', 'docstore', and 'index_to_docstore_id'